## In this version I will integrate a neural netwrok to my reinforcement learning model to enhance its ability to capture complex patterns and relationships in the data thar a linear model might miss. This approach is commonly known as a Deep-Q-Netork (DQN).

In [ ]:
import gurobipy as gp
import pandas as pd
from code_map import final_markets, new_meters, utils, weather, timeframes, rl_utils
import numpy as np
from datetime import datetime, timedelta
from collections import defaultdict, Counter
import random
import sklearn
import torch
import torch.nn as nn
import torch.optim as optim

In [ ]:
tf = timeframes.one_week
L, M, F, H, freq_data, power_meter_dict, consumption_data = utils.get_all_sets(timeframe= tf, areas = ["NO5"])
L_u, L_d, Fu_h_l, Fd_h_l, R_h_l, P_h_m, Vp_h_m, Vm_m, R_m = utils.get_parameters(L = L, M = M, H = H)
Ir_hlm, Ia_hlm, Va_hm = utils.get_income_dictionaries(H=H, L = L, M = M, freq_data= freq_data, Fu_h_l= Fu_h_l, Fd_h_l= Fd_h_l, P_h_m= P_h_m, Vp_h_m= Vp_h_m, F = F, markets_dict = {market.name : market for market in M}, timeframe = tf, areas = ["NO5"])
compatible_dict = utils.get_compatibility_dict(L = L ,M = M, index = False)

In [ ]:
sup_market_names = ["FCR", "aFRR"]
markets = [market for market in M if sup_market_names[0] in market.name  or sup_market_names[1] in market.name]
exp_price_dict, exp_vol_dict = rl_utils.get_expected_prices_and_volumes_dict(bid_timeframe= H, markets= markets)
norm_exp_price_dict = rl_utils.normalize_dict_vals(exp_price_dict, norm_method= "min_max")
norm_exp_vol_dict = rl_utils.normalize_dict_vals(exp_vol_dict, norm_method= "min_max")
weather_data = weather.get_weather_data(tf= tf, areas = ["NO5"])
norm_w_df = rl_utils.normalize_weather_data(weather_data= weather_data)
spot_path = "../master-data/spot_data/spot_june_23.csv"
norm_da_df = final_markets.preprocess_spot_data(pd.read_csv(spot_path), year = tf.year, start_month = tf.start_month, end_month = tf.end_month, start_day = tf.start_day, end_day = tf.end_day, start_hour = tf.start_hour, end_hour = tf.end_hour, area = "NO5", normalize= True)



In [ ]:
markets = [market for market in M if sup_market_names[0] in market.name  or sup_market_names[1] in market.name]
print([m.name for m in markets])

In [ ]:
n_actions = 3
n_features = 9

In [ ]:


# Define the neural network architecture
class QNetwork(nn.Module):
    def __init__(self, input_size, output_size, hidden_sizes):
        super(QNetwork, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_sizes[0])
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_sizes[0], hidden_sizes[1])
        self.output = nn.Linear(hidden_sizes[1], output_size)
        
    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.output(x)
        return x

# Initialize the neural network
input_size = n_features
output_size = n_actions
hidden_sizes = [128, 64] # Example sizes; you can adjust these
q_network = QNetwork(input_size, output_size, hidden_sizes)

# Define the optimizer
optimizer = optim.Adam(q_network.parameters(), lr=0.001)

# During training, collect experiences and store them in a replay buffer
replay_buffer = []

# When it's time to update the model...
batch = random.sample(replay_buffer, batch_size)  # batch_size could be, for example, 32
states, actions, rewards, next_states, dones = zip(*batch)

# Convert to tensors
states = torch.FloatTensor(states)
actions = torch.LongTensor(actions)
rewards = torch.FloatTensor(rewards)
next_states = torch.FloatTensor(next_states)
dones = torch.FloatTensor(dones)

# Compute Q values for current states
current_q_values = q_network(states).gather(1, actions.unsqueeze(1)).squeeze(1)

# Compute Q values for next states using target network
next_q_values = target_network(next_states).max(1)[0].detach()


In [ ]:
target_network = QNetwork(input_size, output_size, hidden_sizes)
target_network.load_state_dict(q_network.state_dict())
# Compute Q values for next states using target network
next_q_values = target_network(next_states).max(1)[0].detach()
targets = rewards + gamma * next_q_values * (1 - dones)  # (1 - dones) to zero out terminal states
# Periodically update target network with weights from the main network
if episode_n % target_update_frequency == 0:
    target_network.load_state_dict(q_network.state_dict())


